# Deep Learning : Computer Go

### Hugo DA COSTA, Tiago FLEURY

# 1. Pre installation et commandes

## Type de GPU :

temps pour epoch de 80 000 samples (batch = 256)

Tesla P100-PCIE-16GB : 130 ms  
Tesla T4 : 240 ms  
Tesla K80 : 538 ms


In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-30297e83-80fd-9354-3339-12897273eb98)


## Installation et telechargement

In [ ]:
!cp /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/project.zip ./

In [ ]:
!unzip project.zip
!mv ./project/* ./

Archive:  project.zip
  inflating: project/Board.h         
  inflating: project/compileMAC.sh   
  inflating: project/compile.sh      
  inflating: project/Game.h          
  inflating: project/games.data      
  inflating: project/golois.cpp      
  inflating: project/golois.cpython-38-x86_64-linux-gnu.so  
  inflating: project/golois.py       
  inflating: project/importGolois.ipynb  
  inflating: project/README          
  inflating: project/Rzone.h         
  inflating: project/test.h5         
  inflating: project/validation.data  


In [ ]:
!pip install pybind11==2.6.0

!pip show pybind11


     |████████████████████████████████| 194kB 14.9MB/s 
Name: pybind11
Version: 2.6.0
Summary: Seamless operability between C++11 and Python
Home-page: https://github.com/pybind/pybind11
Author: Wenzel Jakob
Author-email: wenzel.jakob@epfl.ch
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip uninstall pybind11
!pip install pybind11==2.6.0
!chmod 777 ./compile.sh
!./compile.sh

Uninstalling pybind11-2.6.0:
  Would remove:
    /usr/local/bin/pybind11-config
    /usr/local/lib/python3.7/dist-packages/pybind11-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pybind11/*
Proceed (y/n)? y
  Successfully uninstalled pybind11-2.6.0
  Using cached https://files.pythonhosted.org/packages/92/a5/796ccb611a92cbdf1f6eaa9b2ec9458d5784a7b909efdf8e8afaa3861783/pybind11-2.6.0-py2.py3-none-any.whl
In file included from golois.cpp:17:0:
Board.h: In member function ‘bool Board::isCapturedLadder(int, int, Rzone*)’:
Board.h:1748:8: warning: unused variable ‘n1’ [-Wunused-variable]
    int n1 = nbLiberties (inter, liberties1, stones1, 3);
        ^~
Board.h:1769:12: warning: unused variable ‘n1’ [-Wunused-variable]
        int n1 = nbLiberties (inter, liberties1, stones1, 3);
            ^~
Board.h: In member function ‘void Board::computeLadders(int)’:
Board.h:1800:9: warning: unused variable ‘other’ [-Wunused-variable]
     int other = opponent (color);
         ^~~~~
B

## Loading from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
version_load = "mobileNet26blocks"
scriptname_load = "golois_"+version_load+".py"
filename_load = "hutia-gogo_"+version_load+".h5"

Golois.py

In [ ]:
!cp /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version_load}/{scriptname_load} /content/golois.py

cp: cannot stat '/content/drive/MyDrive/Fac/a M2 IASD/Deep Learning/Projet hutia-gogo/Modèles/MobileV2.1/golois_MobileV2.1.py': No such file or directory


Reseau.h5 + history

In [ ]:
!cp /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version_load}/{filename_load} /content/reseau.h5
!cp /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version_load}/history.json /content/

## Saving to drive

In [ ]:
version = "mobileNet_cazenave_34blocks"
filename = "hutia-gogo_"+version+".h5"
scriptname = "golois_"+version+".py"

Creer Dossier

In [ ]:
!mkdir /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version}

reseau.h5

In [ ]:
!cp reseau.h5 /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version}/{filename}

historique

In [ ]:
!cp history.json /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version}/

In [ ]:
print("done")

done


Archi et Model.png

In [ ]:
!cp archi.py /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version}/


In [ ]:
!cp model.png /content/drive/MyDrive/Fac/"a M2 IASD"/"Deep Learning"/"Projet hutia-gogo"/Modèles/{version}/


# 2. Implémentation

## Imports

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pickle
import json
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.layers import Dense, Conv2D, LeakyReLU, Activation, BatchNormalization, Flatten, Concatenate, GlobalAveragePooling2D, DepthwiseConv2D, ReLU
from keras.utils import plot_model
import golois

ModuleNotFoundError: ignored

## Structure du réseau

In [ ]:
planes = 21

# STRUCTURE DU RESEAU
# INPUT

input = keras.Input(shape=(19, 19, planes), name='board')

# COUCHES

#FIRST LAYER
x = Conv2D(filters=64, kernel_size=1, kernel_regularizer=regularizers.l2(0.0001))(input)
x = BatchNormalization()(x)
x = ReLU()(x)
residu = x

for i in range(26):
  # BLOCK BottleNeck
  #Expansion
  x = Conv2D(256, 1, kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = DepthwiseConv2D(kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  #Squeezing
  x = Conv2D(64, 1, kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
  x = BatchNormalization()(x)

  #Residu
  x = layers.add([x,residu])
  residu = x

#Policy Head
policy_head = Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = Flatten()(policy_head)
policy_head = Activation('softmax', name='policy')(policy_head)

#Value Head
value_head = GlobalAveragePooling2D()(x)
value_head = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = Dense(1, activation='sigmoid', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])



In [ ]:
model.summary()

In [ ]:
plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

## Training

#### Loading
Pour continuer à entrainer un réseau

In [ ]:
# LOAD MODEL
model = keras.models.load_model("reseau.h5")

In [ ]:
# LOAD HISTORY
history = json.load(open('history.json','r'))
old_history = history['train_history']
val_history = history['val_history']

#### Initialisations

In [ ]:
# SI NOUVEAU MODELE
old_history = {
    "loss":[],
    "policy_categorical_accuracy": [],
    "policy_loss": [],
    "value_loss": [],
    "value_mse": []
}

val_history = {
    "val_loss": [],
    "val_policy_categorical_accuracy": [],
    "val_policy_loss": [],
    "val_value_loss": [],
    "val_value_mse": []
}

In [ ]:
N = 100000
planes = 21
moves = 361

In [ ]:
#DATA CONTAINERS
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')


policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
 
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
 
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
 
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')
 
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

getValidation


#### Compilation et Learning Rate

In [ ]:
#COMPILATION ADAM 
model.compile(optimizer=keras.optimizers.Adam(lr=0.005),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

In [ ]:
#COMPILATION SGD  
model.compile(optimizer=keras.optimizers.SGD(lr=0.005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

#### Training Loop

In [ ]:
epochs = 20
batch = 128

In [ ]:
for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1,
                        batch_size=batch)
 
    # Pour garder la trace de l'historique
    for k,v in history.history.items():
      old_history[k].extend(v)

    if (i % 5 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
 
        val_history["val_loss"].append(val[0])
        val_history["val_policy_loss"].append(val[1])
        val_history["val_value_loss"].append(val[2])
        val_history["val_policy_categorical_accuracy"].append(val[3])
        val_history["val_value_mse"].append(val[4])

epoch 1
782/782 [==============================] - 330s 411ms/step - loss: 2.7851 - policy_loss: 1.9978 - value_loss: 0.5781 - policy_categorical_accuracy: 0.4516 - value_mse: 0.2021
epoch 2
782/782 [==============================] - 322s 411ms/step - loss: 2.7925 - policy_loss: 2.0067 - value_loss: 0.5775 - policy_categorical_accuracy: 0.4482 - value_mse: 0.2020
epoch 3
782/782 [==============================] - 321s 411ms/step - loss: 2.7854 - policy_loss: 2.0043 - value_loss: 0.5737 - policy_categorical_accuracy: 0.4486 - value_mse: 0.2004
epoch 4
782/782 [==============================] - 322s 411ms/step - loss: 2.7817 - policy_loss: 2.0008 - value_loss: 0.5744 - policy_categorical_accuracy: 0.4486 - value_mse: 0.2008
epoch 5
782/782 [==============================] - 321s 411ms/step - loss: 2.7752 - policy_loss: 1.9938 - value_loss: 0.5757 - policy_categorical_accuracy: 0.4498 - value_mse: 0.2013
val = [2.8044559955596924, 2.0014426708221436, 0.5975750088691711, 0.4491699934005737

In [ ]:
print("Last val : ",val)

Last val :  [2.7171525955200195, 1.9903762340545654, 0.5596851706504822, 0.4496000111103058, 0.19496023654937744]


#### Saving

Model

In [ ]:
model.save('reseau.h5')

History

In [ ]:
history = {'train_history': old_history,
            'val_history': val_history
            }
with open("history.json","w") as f :
  json.dump(history, f)

# 3. Analyse

#### Imports

In [ ]:
import golois
import math
import numpy as np
import keras
import matplotlib.pyplot as plt
import matplotlib
import pickle
import json
 
COLOR = 'black'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR

Loading model and data

In [ ]:
#model = keras.models.load_model("reseau.h5")
history = json.load(open('history.json','r'))
train_history = history['train_history']
val_history = history['val_history']

In [ ]:
planes = 21
moves = 361
N = 100000
batch = 512

In [ ]:

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')


In [ ]:
golois.getValidation (input_data, policy, value, end)
val = model.evaluate (input_data,
                      [policy, value], verbose = 0, batch_size=batch)